In [ ]:
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns

class HMM:
    def __init__(self, states, observations, start_prob, trans_prob, emit_prob):
        self.states = states               # e.g. Hungry, Full
        self.observations = observations   # e.g. Eat, No Eat
        self.start_prob = start_prob        
        self.trans_prob = trans_prob
        self.emit_prob = emit_prob

    def forward_algorithm(self, observations):
        T = len(observations)
        N = len(self.states)
        forward_prob = np.zeros((N, T))
        scale = np.zeros(T) # Handle numerical underflow

        # Initialization step
        for i, state in enumerate(self.states):
            forward_prob[i, 0] = self.start_prob[state] * self.emit_prob[state][observations[0]]
        # Handle numerical underflow
        scale[0] = np.sum(forward_prob[:, 0])
        forward_prob[:, 0] /= scale[0]

        # Recursion Step
        # Calculate forward probabilities for subsequent observations
        # Consider all possible prev states and their transitions to the current state
        for t in range(1, T):
            for i, curr_state in enumerate(self.states):
                for j, prev_state in enumerate(self.states):
                    forward_prob[i, t] += forward_prob[j, t-1] * self.trans_prob[prev_state][curr_state]
                forward_prob[i, t] *= self.emit_prob[curr_state][observations[t]]
            # Handle numerical underflow
            scale[t] = np.sum(forward_prob[:, t])
            forward_prob[:, t] /= scale[0]

        # Termination step
        total_prob = np.sum(forward_prob[:, -1])

        return forward_prob, total_prob
    
    def viterbi(self, observations):
        T = len(observations)
        N = len(self.states)

        viterbi = np.zeros((N, T))
        backpointer = np.zeros((N, T), dtype=int)
        
        # Initializaiton Step
        for s, state in enumerate(self.states):
            viterbi[s, 0] = (self.start_prob[state] * self.emit_prob[state][observations[0]])
            backpointer[s, 0] = 0

        # Recursion Step
        for t in range(1, T):
            for s, curr_state in enumerate(self.states):
                probs = [viterbi[s_prev, t-1] * self.trans_prob[self.states[s_prev]][curr_state] for s_prev in range(N)]
                best_prev = np.argmax(probs)
                viterbi[s, t] = probs[best_prev] * self.emit_prob[curr_state][observations[t]]
                backpointer[s, t] = best_prev

        # Termination Step 
        best_last_state = np.argmax(viterbi[:, -1])
        best_prob = viterbi[best_last_state, -1]

        path_idx = [best_last_state]
        for t in range(T-1, 0, -1):
            path_idx.insert(0, backpointer[path_idx[0], t])
        
        best_path = [self.states[i] for i in path_idx]
        return best_path, best_prob
    
    def forward_backward(self, observations):

        return A, B

def visualize_forward_probabilities(forward_prob, states, observations):
    plt.figure(figsize=(12, 6))
    sns.heatmap(forward_prob, anot=True, fmt='.2f', cmap='Yl0rRd',
                xticklabels=observations, yticklabels=states)
    plt.title('Forward Probabilities Heatmap')
    plt.xlabel('Observations')
    plt.ylabel('States')
    plt.show()